# Set up environment

In [1]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment
Adding the following directory to the PYTHONPATH: /Users/pauliusztin/Documents/01_projects/hopsworks_recsys/hands-on-recommender-system


# <span style="color:#ff5f27"> Scheduling Hopsworks materialization jobs </span>


## 📝 Imports

In [2]:
from datetime import datetime, timezone

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [3]:
project, fs = hopsworks_integration.get_feature_store()

jobs_api = project.get_jobs_api()

2024-11-20 19:11:47.622 | INFO     | recsys.integrations.hopsworks_integration:get_feature_store:9 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/15551
Connected. Call `.close()` to terminate connection gracefully.


# Retrieving materialization jobs


In [4]:
interactions_job = jobs_api.get_job('interactions_1_offline_fg_materialization')
interactions_job

Job('interactions_1_offline_fg_materialization', 'SPARK')

In [5]:
transactions_job = jobs_api.get_job('transactions_1_offline_fg_materialization')
transactions_job

Job('transactions_1_offline_fg_materialization', 'SPARK')

# Running materialization jobs


In [6]:
interactions_job_execution = interactions_job.run()
interactions_job_execution

Execution started, explore it at https://c.app.hopsworks.ai:443/p/15551/jobs/named/interactions_1_offline_fg_materialization/executions


Execution('UNDEFINED', 'INITIALIZING', '2024-11-20T17:11:50.435Z', '-op offline_fg_materialization -path hdfs:///Projects/pbiuszti/Resources/jobs/interactions_1_offline_fg_materialization/config_1732029249189')

In [7]:
transactions_job_execution = transactions_job.run()
transactions_job_execution

Execution started, explore it at https://c.app.hopsworks.ai:443/p/15551/jobs/named/transactions_1_offline_fg_materialization/executions


Execution('UNDEFINED', 'INITIALIZING', '2024-11-20T17:11:50.696Z', '-op offline_fg_materialization -path hdfs:///Projects/pbiuszti/Resources/jobs/transactions_1_offline_fg_materialization/config_1732029226605')

## <span style="color:#ff5f27">⏰ Scheduling materialization jobs </span>


In [8]:
interactions_job.schedule(
    cron_expression="0 0 0 * * ?",  # Runs at midnight (00:00:00) every day
    start_time=datetime.now(tz=timezone.utc),
)
interactions_job.job_schedule.next_execution_date_time

datetime.datetime(2024, 11, 21, 0, 0, tzinfo=datetime.timezone.utc)

In [9]:
transactions_job.schedule(
    cron_expression="0 0 0 * * ?",  # Runs at midnight (00:00:00) every day
    start_time=datetime.now(tz=timezone.utc),
)
transactions_job.job_schedule.next_execution_date_time

datetime.datetime(2024, 11, 21, 0, 0, tzinfo=datetime.timezone.utc)

## <span style="color:#ff5f27"> Inspecting the materialization jobs in Hopsworks UI </span>

View results in [Hopsworks Serverless](https://rebrand.ly/serverless-github): **Compute → Ingestions**